In [1]:
fo=open("chatbot.txt","r")
chatbot_data=fo.read()
fo.close()

In [2]:
list_of_sentences=chatbot_data.replace("\t"," ").split("\n")

In [3]:
import re
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
def ngram_to_list(sentences):
    corpus=[]
    count_is=0
    for i in sentences:
        review=re.sub("[^a-zA-Z]"," ",i)
        corpus.append(review)
        if len(i)>count_is:
            count_is=len(i)
    cv=CountVectorizer(binary=True,ngram_range=(1,count_is))
    X=cv.fit_transform(corpus)
    list_ngrams=list(cv.vocabulary_)
    return list_ngrams


In [5]:
corpus_of_normal_text=ngram_to_list(list_of_sentences)
print(len(corpus_of_normal_text))

90834


In [6]:
import pandas as pd

# Creating dataframe of normal text

In [7]:
df_chat={}
df_chat["Response"]=corpus_of_normal_text
df_chat["Label"]=[0 for i in range(len(corpus_of_normal_text))]
df_chat=pd.DataFrame(df_chat)

In [8]:
print(df_chat.shape)
print("total no of unique values are :")
print(df_chat.nunique())
df_chat.head()

(90834, 2)
total no of unique values are :
Response    90834
Label           1
dtype: int64


,Response,Label
0,what,0
1,are,0
2,your,0
3,interests,0
4,am,0


# Creating dataframe of gibberish text

In [9]:
df_gibberish=pd.read_csv("Gibberish.csv")

In [10]:
print(df_gibberish.shape)
print("total no of unique values are :")
print(df_gibberish.nunique())
df_gibberish.head()

(3767, 2)
total no of unique values are :
Response    3516
Label          1
dtype: int64


,Response,Label
0,ggg,1
1,hgghghghghghghhg,1
2,ufdhgjndfnvbhfdjvnjkmfgbdfg,1
3,dbdbdbd,1
4,dfgdfgd,1


In [11]:
list_of_gibberish=list(df_gibberish["Response"])

In [12]:
corpus_of_gibberish_text=ngram_to_list(list_of_gibberish)
print(len(corpus_of_gibberish_text))

23239


In [13]:
df_gibberish={}
df_gibberish["Response"]=corpus_of_gibberish_text
df_gibberish["Label"]=[1 for i in range(len(corpus_of_gibberish_text))]
df_gibberish=pd.DataFrame(df_gibberish)

In [14]:
print(df_gibberish.shape)
print("total no of unique values are :")
print(df_gibberish.nunique())
df_gibberish.head()

(23239, 2)
total no of unique values are :
Response    23239
Label           1
dtype: int64


,Response,Label
0,ggg,1
1,hgghghghghghghhg,1
2,ufdhgjndfnvbhfdjvnjkmfgbdfg,1
3,dbdbdbd,1
4,dfgdfgd,1


# combining df_chat and df_gibberish

In [15]:
df_main=pd.concat([df_gibberish,df_chat],ignore_index=True)

In [16]:
print(df_main.shape)
print("total no of unique values are :")
print(df_main.nunique())
df_main.head()

(114073, 2)
total no of unique values are :
Response    113989
Label            2
dtype: int64


,Response,Label
0,ggg,1
1,hgghghghghghghhg,1
2,ufdhgjndfnvbhfdjvnjkmfgbdfg,1
3,dbdbdbd,1
4,dfgdfgd,1


In [17]:
df_main.tail()

,Response,Label
114068,use sense similar to sonar to determine the lo...,0
114069,sense similar to sonar to determine the locati...,0
114070,dolphins use sense similar to sonar to determi...,0
114071,use sense similar to sonar to determine the lo...,0
114072,dolphins use sense similar to sonar to determi...,0


# converting of text to tokens

In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [19]:
vocab=100000

In [30]:
sentences=[]
for i in list(df_chat["Response"]):
    review=re.sub("[^a-zA-Z]"," ",i)
    review=review.split()
    review=[i for i in review if i not in sentences and len(i)>1]
    sentences.extend(review)
len(sentences)

1854

In [31]:
fo=open("paragraph.txt","r")
fo_text=fo.read()
fo.close()

In [32]:
for i in fo_text.split():
    i=re.sub("[^a-zA-Z]","",i)
    if i not in sentences and len(i)>1:
        sentences.append(i)
len(sentences)

3920

In [34]:
tokenizer=Tokenizer(num_words=vocab,oov_token="<OOV>")

In [35]:
tokenizer.fit_on_texts(sentences)

In [36]:
len(tokenizer.word_index)

3618

In [37]:
sequences=tokenizer.texts_to_sequences(df_main["Response"])

In [38]:
max_len=0
for i in sequences:
    if len(i)>max_len:
        max_len=len(i)
print(max_len)

69


In [39]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [40]:
padded=pad_sequences(sequences,maxlen=max_len)

In [45]:
padded

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    0,    1],
       ...,
       [   0,    0,    0, ...,    3, 1956, 1957],
       [   0,    0,    0, ..., 1956, 1957, 1958],
       [   0,    0,    0, ..., 1956, 1957, 1958]])

# splitting training and test data

In [46]:
from sklearn.model_selection import train_test_split

In [48]:
X_train, X_test, y_train, y_test = train_test_split(padded, df_main["Label"], test_size=0.40,random_state=0)

In [49]:
print("the shape of X_train is : ",X_train.shape)
print("the shape of X_test is : ",X_test.shape)
print("the shape of y_train is : ",y_train.shape)
print("the shape of y_test is : ",y_test.shape)

the shape of X_train is :  (68443, 69)
the shape of X_test is :  (45630, 69)
the shape of y_train is :  (68443,)
the shape of y_test is :  (45630,)


In [50]:
print(y_test.value_counts())

0    36313
1     9317
Name: Label, dtype: int64


In [51]:
import keras

In [52]:
model = keras.models.Sequential([
    keras.layers.Embedding(vocab, 64),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(rate=0.25),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dropout(rate=0.25),
    keras.layers.Dense(1, activation="sigmoid")
])

In [53]:
model.compile(optimizer="adam",loss='binary_crossentropy',metrics=["acc"])

In [54]:
history = model.fit(X_train, y_train, epochs=1, validation_data=(X_test, y_test),  verbose=1)

2139/2139 [==============================] - 309s 143ms/step - loss: 0.0522 - acc: 0.9819 - val_loss: 0.0099 - val_acc: 0.9982


In [55]:
y_predict=model.predict(X_test)

1426/1426 [==============================] - 4s 3ms/step


In [56]:
y_predict=y_predict>0.5

In [57]:
from sklearn.metrics import accuracy_score,classification_report

In [58]:
accuracy_score(y_predict,y_test)

0.9982467674775367

In [59]:
print(classification_report(y_predict,y_test))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00     36393
        True       0.99      1.00      1.00      9237

    accuracy                           1.00     45630
   macro avg       1.00      1.00      1.00     45630
weighted avg       1.00      1.00      1.00     45630



# testing the text

1. named entities cant be recognized
2. spelling mistakes are taken as gibberish
3. shortcuts cant be predicted
4. mixture of gibberish and normal text classification is sometimes unpredictable it depends on maximum no of words
5. symbols are taken as normal text but by adding additional data of symbols to dataframe can help the model to predict
6. it will not work for multi class classification

In [103]:
def testing_text(text):
    dummy=[]
    dummy.append(text)
    sequences=tokenizer.texts_to_sequences(dummy)
    print(sequences)
    padded=pad_sequences(sequences,maxlen=max_len)
    print(padded)
    y=model.predict(padded)
    print("predicted value :",y)
    y=y>0.5
    if y:
        print("text is gibberish")
    else:
        print("text is normal")

In [104]:
testing_text("hello morning how are you doing")

[[1794, 1034, 58, 293, 25, 782]]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0 1794 1034   58  293   25  782]]
1/1 [==============================] - 0s 143ms/step
predicted value : [[8.567931e-06]]
text is normal


In [105]:
testing_text("hello")

[[1794]]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0 1794]]
1/1 [==============================] - 0s 177ms/step
predicted value : [[0.00441073]]
text is normal


In [106]:
testing_text("hlo")

[[1]]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]
1/1 [==============================] - 0s 117ms/step
predicted value : [[0.98527795]]
text is gibberish


In [107]:
testing_text("$#$%$^")

[[]]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
1/1 [==============================] - 0s 159ms/step
predicted value : [[0.00560622]]
text is normal


In [108]:
testing_text("()()()")

[[]]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
1/1 [==============================] - 0s 159ms/step
predicted value : [[0.00560622]]
text is normal


In [109]:
testing_text("abc()()()")

[[1]]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]
1/1 [==============================] - 0s 116ms/step
predicted value : [[0.98527795]]
text is gibberish


In [110]:
testing_text("jcjcj bdvd b khjbd hsdbc")

[[1, 1, 1, 1, 1]]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1]]
1/1 [==============================] - 0s 119ms/step
predicted value : [[1.]]
text is gibberish


In [111]:
testing_text("aq()()()")

[[1]]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]
1/1 [==============================] - 0s 119ms/step
predicted value : [[0.98527795]]
text is gibberish


In [112]:
testing_text("good morning gg")

[[63, 1034, 1]]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0   63 1034    1]]
1/1 [==============================] - 0s 164ms/step
predicted value : [[0.99620086]]
text is gibberish


In [113]:
testing_text("gg good morning")

[[1, 63, 1034]]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    1   63 1034]]
1/1 [==============================] - 0s 120ms/step
predicted value : [[0.99620086]]
text is gibberish


In [114]:
testing_text("qq hello hi how")

[[1, 1794, 1795, 58]]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    1 1794 1795   58]]
1/1 [==============================] - 0s 265ms/step
predicted value : [[0.5238035]]
text is gibberish


In [115]:
testing_text("qq hello hi how are")

[[1, 1794, 1795, 58, 293]]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    1 1794 1795   58  293]]
1/1 [==============================] - 0s 167ms/step
predicted value : [[0.01068687]]
text is normal


In [116]:
testing_text("prem")

[[1]]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]
1/1 [==============================] - 0s 143ms/step
predicted value : [[0.98527795]]
text is gibberish


In [117]:
testing_text("hello my name is prem")

[[1794, 19, 81, 23, 1]]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0 1794   19   81   23    1]]
1/1 [==============================] - 0s 121ms/step
predicted value : [[0.01379605]]
text is normal


In [118]:
testing_text("pune")

[[1]]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]
1/1 [==============================] - 0s 110ms/step
predicted value : [[0.98527795]]
text is gibberish


In [119]:
testing_text("my company is located in pune")

[[19, 1882, 23, 1, 16, 1]]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   19 1882   23    1   16    1]]
1/1 [==============================] - 0s 180ms/step
predicted value : [[0.99619013]]
text is gibberish


In [120]:
testing_text("my company is in pune")

[[19, 1882, 23, 16, 1]]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   19 1882   23   16    1]]
1/1 [==============================] - 0s 114ms/step
predicted value : [[0.0111645]]
text is normal
